In [1]:
import os
from fastai.text import * 

In [2]:
torch.cuda.device_count()

1

In [3]:
LANG = 'en'

In [4]:
proj_path = Path(os.getcwd()).parents[1]

In [5]:
data_path = proj_path/f'clean_data_csv_{LANG}'

In [6]:
df = pd.read_csv(data_path/'all_gender_data.csv')

In [7]:
df.head()

,gender,text
0,male,@ChadPergram Can he call his mom?
1,male,@birdonawireart ended up choosing a bear eatin...
2,female,RT @IntJudoFed: @HeymanHustle @WWE @ufc @Brock...
3,male,RT @allyshepp: y’all ... this is how my little...
4,female,"RT @Yaqiu: Per @rockngo , Chinese police haras..."


In [8]:
data_lm = TextLMDataBunch.from_csv(data_path, 'all_gender_data.csv')

In [9]:
data_clas = TextClasDataBunch.from_csv(data_path, 'all_gender_data.csv', 
                                       vocab=data_lm.train_ds.vocab, bs=32)

In [10]:
data_lm.save(f'data_lm_export_{LANG}.pkl')
data_clas.save(f'data_clas_export_{LANG}.pkl')

In [10]:
data_lm = load_data(data_path, fname=f'data_lm_export_{LANG}.pkl')
data_clas = load_data(data_path, fname=f'data_clas_export_{LANG}.pkl', bs=16)

In [12]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,3.515250,3.248378,0.462042


In [13]:
learn.unfreeze()
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy
1,3.054218,2.920586,0.509641


In [14]:
learn.predict("This is a review about", n_words=10)

'This is a review about the History of David Gates http :'

In [15]:
learn.save_encoder('ft_enc')

In [11]:
learn_class = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5)
learn_class.load_encoder('ft_enc')

In [12]:
data_clas.show_batch()

text,target
xxbos @pizster @bendleshnitz xxup proud ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤ ️ ❤,human
"xxbos xxmaj senior xxmaj web xxmaj developer – xxmaj global xxup it ( xxmaj boise , xxup id ) : xxmaj job xxmaj title : xxmaj senior xxmaj web xxmaj developer – xxmaj global xxup it ( xxmaj boise , xxup id ) xxmaj location : xxmaj boise , xxup id , xxmaj west xxmaj united xxmaj states , xxup usa xxmaj company : xxup xxunk xxmaj xxunk xxmaj company",bot
"xxbos xxmaj mechanical xxmaj engineer : xxmaj glendale , xxup ca xxmaj full - time $ 90k – $ 110k xxmaj posted xxmaj today xxmaj if you are a xxmaj mechanical xxmaj engineer with autocad experience , please read on ! xxmaj top xxmaj reasons to xxmaj work with xxmaj us – xxmaj competitive xxmaj base xxmaj salary – xxmaj great xxmaj benefits xxmaj what xxmaj you xxmaj will xxmaj",bot
"xxbos xxmaj network xxmaj engineer : xxmaj employee xxmaj type : xxmaj full - xxmaj time xxmaj location : xxmaj lexington , xxup ma xxmaj job xxmaj type : xxmaj engineering xxmaj experience : xxmaj not xxmaj specified xxmaj date xxmaj posted : 8 / 8 / 2018 xxmaj about xxmaj us xxmaj about xxmaj us xxmaj at xxmaj booz xxmaj allen , we harness our collective ingenuity to solve",bot
"xxbos .net xxmaj developer : .net xxmaj developer xxmaj ref xxmaj no.:18 - xxunk xxmaj location : xxmaj boston , xxmaj massachusetts xxmaj position xxmaj type : xxmaj full xxmaj time / xxmaj contract xxmaj start xxmaj date : 07 / 25 / 2018 .net with xxup mvc , xxmaj web xxup api , xxmaj lin q xxmaj position xxmaj responsibilities : – xxmaj must have expert - level expertise",bot


In [13]:
learn_class.fit_one_cycle(3, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,0.428252,0.279258,0.883484
2,0.417792,0.267365,0.890232
3,0.344555,0.242927,0.901203
